# Neuron, input, synapse

## Neuron model
In this session, we will move from a biological neuron to a simpler phenomenological neuron in which we retain the key features of a neuron i.e. spikes and spike patterns. 

We will use a neuron model called the adaptive leaky-and-integrate firing neuron. Using this neuron you will get more insights into the origin of different types of spikes patterns.

In [ ]:
#@title Run the following to initialize lab environment.

!pip install ipympl ipywidgets stg-net -q

import matplotlib.pyplot as plt         # import matplotlib
from matplotlib.widgets import Slider
import numpy as np                      # import numpy
import ipywidgets as widgets            # interactive display

# Colab setting for widget
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError:
    pass

# modeling library
from stg_net.neuron import LIF
from stg_net.input import Poisson_generator, Gaussian_generator, Current_injector
from stg_net.conn import Simulator
from stg_net.helper import plot_volt_trace

# setting for figures
fig_w, fig_h = 8, 6
my_fontsize = 18
my_params = {'axes.labelsize': my_fontsize,
          'axes.titlesize': my_fontsize,
          'figure.figsize': (fig_w, fig_h),
          'font.size': my_fontsize,
          'legend.fontsize': my_fontsize-4,
          'lines.markersize': 8.,
          'lines.linewidth': 2.,
          'xtick.labelsize': my_fontsize-2,
          'ytick.labelsize': my_fontsize-2}
plt.rcParams.update(my_params)
my_layout = widgets.Layout()

# Auto Reloading
%load_ext autoreload
%autoreload 2

# Widget interaction
%matplotlib widget

### Firing patterns of a single neuron (15min)

Here we have a simple neuron model with 5 parameters. Two of them are just the parameters for LIF neuron and three of them captures the adaptation behavior. In addition to these parameters, the input level of current injection can be changed. So we essentially 6 parameters to control this single neuron.   

John von Neumann famously said that with four parameters I can fit an elephant and with six I can make it wiggle its trunk. Well, let's see what we can do with six parameters in terms of generating biologically plausible firing patterns of a neuron.

In [ ]:
#@title Run the following to start the simulation of single neuron { vertical-output: true }
T, dt = 5e2, 0.1    # simulation period(ms), step size(ms)
wt, dl = 1., 5.
rt = 65.

# simualtor
h = Simulator(dt=dt)

# neurons
nrn = LIF(sim=h)
nrn.g_L = 2.
nrn.E_L = -70.
nrn.V_th = -40.

tonic_neuron = {'tau_m':20., 'a':0., 'tau_w':30., 'b':3., 'V_reset':-55.}
adapting_neuron = {'tau_m':20., 'a':0., 'tau_w':100., 'b':0.5, 'V_reset':-55.}
initburst_neuron = {'tau_m':10., 'a':0., 'tau_w':100., 'b':1., 'V_reset':-50.}
bursting_neuron = {'tau_m':5., 'a':0., 'tau_w':100., 'b':1., 'V_reset':-46.}
irregular_neuron = {'tau_m':10., 'a':-0.01, 'tau_w':50., 'b':1.2, 'V_reset':-46.}
transient_neuron = {'tau_m':5., 'a':0.05, 'tau_w':100., 'b':0.7, 'V_reset':-60.}
delayed_neuron = {'tau_m':5., 'a':-0.1, 'tau_w':100., 'b':1., 'V_reset':-60.}

neuron_params = {'tonic_neuron': tonic_neuron, 'adapting_neuron': adapting_neuron, 
                 'initburst_neuron': initburst_neuron, 'bursting_neuron': bursting_neuron, 
                 'irregular_neuron': irregular_neuron, 'transient_neuron': transient_neuron, 
                 'delayed_neuron': delayed_neuron, 'my_neuron': tonic_neuron}
# tonic
nrn.tau_m = 20.
nrn.tau_w = 30.
nrn.b = 3
nrn.V_reset = -55.

# current injection
cur_in = Current_injector(sim=h, rate=rt, start=int(T/dt*0.1), end=int(T/dt*0.9))
nrn.connect(cur_in, {'ctype':'Static', 'weight': wt, 'delay':dl})

# updating parameters
def update(Icur=65., tau_m=20., V_reset=-55., a=0., tau_w=30., b=0., neuron_type='my_neuron'):
    if neuron_type == 'my_neuron':
      neuron_params['my_neuron'] = {'tau_m':tau_m, 'a':a, 'tau_w':tau_w, 'b':b, 'V_reset':V_reset}
    nrn.update(neuron_params[neuron_type])

    # constant current injection
    cur_in.rate = Icur

    # simulate
    h.run(T)

    # visualize
    plt.clf()
    plt.title('firing patterns')
    plt_par = {'dt':dt, 'range_t':np.arange(0., T, dt), 'V_th':nrn.V_th}
    plot_volt_trace(plt_par, nrn.v, np.array(nrn.spikes['times']))
    plt.xlim([0, 350])
    plt.ylim([-80, -20])
    plt.tight_layout()

try:
  plt.figure(fig_nrn)
  plt.clf()
except:
  print('Init figure')
fig_nrn = plt.figure()
widgets.interact(update, neuron_type=neuron_params.keys(), tau_m=(5.0, 20.0, 1.0), a=(-0.1, 0.1, 0.01), tau_w=(30., 120., 10.), b=(0., 3., 0.1), V_reset=(-60., -40., 2.), Icur=(5., 70., 5.));

## Describe
- How are the parameters Icur, V_reset, τm affecting the neuron’s activity?
- How are the adaptation parameters a,b,τw affecting the neuron’s activity?
- Can you tune all the six parameters to find some unique firing patterns? 
- What are the most important variables of the model, in the sense that they change the firing pattern in a qualitatively different manner e.g. from regular spiking to bursting?

## Bonus
- Which other spike patterns can you imagine?

## Think
- We often try to describe a neuron with its input current and output firing rate curve. Can we define such a relationship for a neuron when it is either bursting or showing firing rate adapataion. If not, what can be done to describe the neuron's behavior?

### Equations
Neuron dynamaics: let's open the balckbox of the neuron

$$\tau_m \frac{dV}{dt} = -(V-V_{rest}) + \Delta_T exp(\frac{V-V_{rh}}{\Delta_T}) - Rw + RI(t)$$
$$\tau_w \frac{dw}{dt} = a(V-V_{rest}) -w + b
\tau_w \sum_{t^{(f)}} \delta(t-t^{(f)})$$

It's described by two variables $V$ and $w$ which denote the membrane potential and adaptation current. The parameters $a, b, \tau_w$ controls the behavior of adaptation where $a$ controls sub-threshold adaptation and $b, \tau_w$ controls spike adaptation. For detailed explanation of this mode, refer tht book we mentioned at the beginning.

## Synapse model (5min)

In the following we connected two neurons with an excitatory synapses. The synapse can be 


*   Static: The synaptic weight (amplitude) remains constant irrespective of the input rate
*   Short-term facilitation: The synaptic weight increases with every spike. The increase is dependent on the firing rate of the pre-synaptic neuron.
*   Short-term depression: The synaptic weight decreases with every spike. The decrease is dependent on the firing rate of the pre-synaptic neuron.

Finally, we have a gap junction that pre-synaptic neuron and post-synaptic neuron are electricially coupled.

In [ ]:
#@title Run the following to start synapse simulation { vertical-output: true }
rt = 100.
N = 2                  # number of neurons
    
# input types
Itypes = ['Icur', 'Gaussian', 'Poisson']

# connection types
Ctypes = ['Static', 'Faci', 'Depr', 'Gap'] 

# updating parameters
def update_syn(c_ie='Static'):   
    # simualtor
    h = Simulator(dt=dt)  

    # network of neurons
    nrns = [LIF(sim=h) for _ in range(N)]

    nrns[0].update(tonic_neuron)
    nrns[1].update(tonic_neuron)

    # background noise
    noise = Current_injector(sim=h, rate=rt, start=int(T/dt*0.1), end=int(T/dt*0.9))
    nrns[0].connect(noise, {'ctype':'Static', 'weight':wt, 'delay':dl})

    # recurrent connections
    tps = [['Static']*N]*N
    J_ei = 0.
    tps[1][0] = c_ie
    if c_ie == 'Gap':
        J_ie = J_ei = 10
        tps[0][1] = tps[1][0] = c_ie
    elif c_ie == 'Static':
        J_ie = 1.0
    elif c_ie == 'Faci':
        J_ie = 1.0
    elif c_ie == 'Depr':
        J_ie = 1.0
    con = np.array([[0., J_ei],
                    [J_ie, 0.]])
    dly = np.random.uniform(2., 5., (N,N))
    synspecs = [[{} for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            synspecs[i][j] = {'ctype':tps[i][j], 'weight':con[i,j], 'delay':dly[i,j]}
    h.connect(nrns, nrns, synspecs)

    # simulation
    h.run(T)

    # coincidence
    binwindow = int(5.0/dt)
    spike_trains = [nrn.states for nrn in nrns]
    bin_spikes = [np.convolve(strain, np.ones(binwindow), 'same') for strain in spike_trains]
    deltats = np.linspace(-10., 10., 21)
    coins = []
    for delay in deltats:
        index = int(delay/dt)
        if index > 0:
            coins.append(np.dot(bin_spikes[0][:-index], bin_spikes[1][index:]))
        elif index < 0:
            coins.append(np.dot(bin_spikes[1][:index], bin_spikes[0][-index:]))
        else:
            coins.append(np.dot(bin_spikes[0], bin_spikes[1]))
    with np.errstate(divide='ignore', invalid='ignore'):
        coins = np.divide(np.array(coins), np.sqrt(np.dot(bin_spikes[0], bin_spikes[0]))*np.sqrt(np.dot(bin_spikes[1], bin_spikes[1])))

    # visualize
    plt.clf()
    cs = ['b', 'r']
    plt.subplot(2,N,1)
    plt.title('raster')
    for nrn, c, l in zip(nrns, cs, range(N)):
        plt.eventplot(nrn.spikes['times'], lineoffsets=2*l, colors=c, label='%dHz'%(len(nrn.spikes['times'])/T*1e3))
    plt.xlabel('Time(ms)')
    plt.yticks([0, 2], ['N_src', 'N_tar'])
    plt.xlim([0., T])
    plt.legend()

    # plt.subplot(2,N,2)
    # plt.title('spike correlation')
    # plt.plot(deltats, coins)
    # plt.xlabel(r'$\delta t$(ms)')
    # plt.ylabel('spike corr')
    # plt.ylim([0, 1])

    # voltage traces
    plt.subplot(2,N,N+1)
    for id, c in zip(range(N), cs):
        if id == 0:
            plt.title('voltage trace')
        plt_par = {'dt':dt, 'range_t':np.arange(0., T, dt), 'V_th':nrns[id].V_th}
        plot_volt_trace(plt_par, nrns[id].v, np.array(nrns[id].spikes['times']), c=c)
        plt.xlim([0., T])

    # # gap junction
    # plt.subplot(2,N,N+2)
    # plt_par = {'dt':dt, 'V_ths':[nrn.V_th for nrn in nrns]}
    # pair_volt_trace(plt_par, [nrn.v for nrn in nrns], [np.array(nrn.spikes['times']) for nrn in nrns])

    plt.tight_layout()

try:
  plt.figure(fig_syn)
  plt.clf()
except:
  print('Init figure')

fig_syn, axes = plt.subplots(2,N,figsize=(5*N,8))
widgets.interact(update_syn, c_ie=Ctypes);

## Describe
- What do you notice about lag (time delay) when comparing chemical synapse types with gap junctions?

- Biologically, synapses are not fixed. What short-term dynamics can you observe?

## Think
- If synapses change all the time, what constitutes a stable network? How variable should the synapses be in long term?

## Input activities (5min)

We'll model the input activities with three simple forms. Think about their biological relavence.
- Constant current injection which models the traditional injection experiment
- Gaussian background noise which models the fluctuation of noisy inputs
- Spike train which models the signal transmission from presynaptic neuron to post-synaptic neuron

In [ ]:
#@title Run the following to start input simulation { vertical-output: true }

# updating parameters
def update_inp(Itype='Icur'):
    # simualtor
    h = Simulator(dt=dt)

    # background noise
    if Itype == 'Icur':
        noise = Current_injector(sim=h, rate=50.0, start=int(T/dt*0.1), end=int(T/dt*0.9))
    elif Itype == 'Gaussian':
        noise = Gaussian_generator(sim=h, mean=0., std=50.0)
    elif Itype == 'Poisson':
        noise = Poisson_generator(sim=h, rate=50.0)
    else:
        print('Invalid input')

    h.run(T)

    # visualize
    plt.clf()
    if Itype == 'Icur':
        noise = plt.plot(noise.Is)
        plt.xlabel('time (ms)')
        plt.ylabel('I (nA)')
    elif Itype == 'Gaussian':
        noise = plt.plot(noise.Is)
        plt.xlabel('time (ms)')
        plt.ylabel('I (nA)')
    elif Itype == 'Poisson':
        noise = plt.plot(noise.spike_train)
        plt.xlabel('time (ms)')
        plt.ylabel('spike')
    
    plt.tight_layout()

try:
  plt.figure(fig_inp)
  plt.clf()
except:
  print('Init figure')

fig_inp = plt.figure()
widgets.interact(update_inp, Itype=['Icur', 'Gaussian', 'Poisson']);

## Bonus
- How would you model the input activities computationally?
- How would you measure the input activities experimentally?